# K_omega vs K_eps : Pb_Hydaulique

In [ ]:
from trustutils import run
run.introduction("Elie Saikali & Alan Burlot")
run.TRUST_parameters()

In [ ]:
run.reset()
run.initBuildDirectory()

dis = [("VDF", ""), ("VEF", "trianguler_H dom")]

for d, dd in dis:
    run.addCaseFromTemplate("jdd-kepsilon.data", f"{d}/KEPS", {"dis" : d, "tri":dd })

# TODO later : VDF komega
run.addCaseFromTemplate("jdd-komega.data", f"VEF/KOMEGA", {"dis" : "VEF", "tri":"trianguler_H dom" })  

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot
a = plot.Graph("Residuals")

for d, _ in dis:
    a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/KEPS/jdd-kepsilon.dt_ev",label=f"{d}_KEPS")
    
a.addResidu(f"{run.BUILD_DIRECTORY}/VEF/KOMEGA/jdd-komega.dt_ev",label=f"VEF_KOMEGA")

a.scale(yscale='log')

In [ ]:
a = plot.Graph("Pressure")
for d, _ in dis:
    a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/KEPS/jdd-kepsilon_PRESSION.son", label=f"{d}_KEPS")

a.addSegment(f"{run.BUILD_DIRECTORY}/VEF/KOMEGA/jdd-komega_PRESSION.son", label=f"VEF_KOMEGA")


In [ ]:
a = plot.Graph("Velocity X")
for d, _ in dis:
    a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/KEPS/jdd-kepsilon_VITESSE.son", label=f"{d}_KEPS")

a.addSegment(f"{run.BUILD_DIRECTORY}/VEF/KOMEGA/jdd-komega_VITESSE.son", label=f"VEF_KOMEGA")

